In [9]:
from astropy.coordinates import SkyCoord, Galactocentric, Galactic
import astropy.units as u
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit,minimize
import emcee
import corner
star_list = fits.open("near_Sun_stars_08.fits")#,memmap=True)
# print(star_list[1].columns)
data=star_list[1].data
l_list = data['l']
ra_i=data['ra']*u.deg
dec_i=data['dec']*u.deg
distance_i=1/data['parallax']*u.kpc
# define Skycoord 
skycoords = SkyCoord(ra =ra_i, dec = dec_i,\
                     pm_ra_cosdec=data['pmra']*u.mas/u.yr, pm_dec=data['pmdec']*u.mas/u.yr, \
                      radial_velocity=data['radial_velocity']*u.km/u.s, \
                     frame='icrs')

In [17]:
pmra = data['pmra']
pmdec = data['pmdec']
pmra_error = data['pmra_error']
pmdec_error = data['pmdec_error']
ra = data['ra']
dec = data['dec']
alphaG = np.radians(192.85948)
deltaG = np.radians(27.12825)
lNGP = np.radians(122.93192)

C1 = np.sin(deltaG)*np.cos(np.radians(dec))-np.cos(deltaG)*np.sin(np.radians(dec))*np.cos(np.radians(ra)-alphaG)
C2 = np.cos(deltaG)*np.sin(np.radians(ra)-alphaG)
cosb = 1/np.sqrt(C1**2+C2**2)
pm_l = 1/cosb*(C1*pmra+C2*pmdec)
pm_l_error = 1/cosb*(C1*pmra_error+C2*pmdec_error)
pm_l_error = pm_l_error*4.74047#/ u.mas * u.yr 
# convert to km s-1 kpc-1, and make it unitless
print(pm_l_error[:10])

[1.5892938  0.03542032 0.04819077 0.05894655 0.07123122 0.03412306
 0.06807578 0.05169063 0.08214416 0.06118468]


In [15]:
l_bin = np.arange(0,361,1)
sample_n = len(data['l'])                                         
error_binned = [[] for _ in range(360)]
for i in np.arange(0,sample_n,1):
    l = l_list[i]
    for j in l_bin:
        if l >= l_bin[j] and l < l_bin[j+1]:
#             print(miu_l_corrected[i]/u.km*u.s*u.kpc)
            error_binned[j].append(pm_l_error[i])
print(i)

306872


In [16]:
binned_mean_error = list(map(lambda i: np.mean(i), error_binned))

print(binned_mean_error[:5])

[0.7239787016538144, 0.7381434967481327, 0.6994645958872041, 0.6664861082116326, 0.6249150124143048]
